In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from PIL import Image
import os


df = pd.read_csv('written_name_validation_v2.csv')

# Display the first few rows of the DataFrame to understand its structure
print(df.head())

              FILENAME    IDENTITY
0  VALIDATION_0001.jpg       BILEL
1  VALIDATION_0002.jpg  LAUMIONIER
2  VALIDATION_0003.jpg         LEA
3  VALIDATION_0004.jpg   JEAN-ROCH
4  VALIDATION_0005.jpg        RUPP


In [ ]:
from PIL import Image
import os

dataset_folder = 'dataset'
image_data = []
labels = []

# Load only the first 90 rows
df = df.head(90)

# Load Images and Labels
dataset_folder = 'dataset'
image_data = []
labels = []

for index, row in df.iterrows():
    image_path = os.path.join(dataset_folder, row['FILENAME'])
    img = Image.open(image_path).convert('L')  # Convert to grayscale
    img = img.resize((64, 64))  # Resize images if needed
    img_array = np.array(img)
    image_data.append(img_array)
    labels.append(row['IDENTITY'])

# Convert lists to NumPy arrays
x_images = np.array(image_data)
y_labels = np.array(labels)

# Preprocess Data
# Normalize image data and handle variations in lighting and shade
x_images = x_images / 255.0
x_images = (x_images - np.mean(x_images)) / np.std(x_images)

# Convert labels to numerical format
label_encoder = LabelEncoder()
y_labels_encoded = label_encoder.fit_transform(y_labels)

# Split Data into Training and Validation Sets
x_train, x_val, y_train, y_val = train_test_split(x_images, y_labels_encoded, test_size=0.2, random_state=42)

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
print(x_train.shape)



(72, 64, 64)


In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x_images, y_labels_encoded, test_size=0.2, random_state=42)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

# Build and Train the Model
model = tf.keras.models.Sequential([
    tf.keras.layers.Reshape((64, 64, 1), input_shape=(64, 64)),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dense(units=len(np.unique(y_labels_encoded)), activation='softmax')
])



In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])



In [ ]:
model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=10)